# Propositionalization: Robot sensor data

In this notebook, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.


Summary:

- Prediction type: __Regression__
- Domain: __Robotics__
- Prediction target: __The force vector on the robot's arm__ 
- Population size: __15001__

# Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

The data set has been generously provided by Erik Berger who originally collected it for his dissertation:

> Berger, E. (2018). *Behavior-Specific Proprioception Models for Robotic Force Estimation: A Machine Learning Approach.* Freiberg, Germany: Technische Universitaet Bergakademie Freiberg.

## A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

# Analysis

## Table of contents

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)

We begin by importing the libraries and setting the project.

In [1]:
import datetime
import os
import sys
import time
from urllib import request

import getml
import getml.data as data
import getml.data.roles as roles
import getml.database as database
import getml.engine as engine
import getml.feature_learning.aggregations as agg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

plt.style.use("seaborn")
%matplotlib inline

getml.engine.set_project("robot")




Connected to project 'robot'


In [2]:
sys.path.append(os.path.join(sys.path[0], ".."))

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

## 1. Loading data

### 1.1 Download from source


In [3]:
data_all = getml.data.DataFrame.from_csv(
    "https://static.getml.com/datasets/robotarm/robot-demo.csv", "data_all"
)

In [4]:
data_all

name              3              4              5              6   ...            105            106            f_x
 role   unused_float   unused_float   unused_float   unused_float   ...   unused_float   unused_float   unused_float
    0         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -11.03 
    1         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.848
    2         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.666
    3         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.507
    4         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.413
             ...            ...            ...            ...                   ...            ...            ...   
14996         3.0837        -0.8836         1.4501        -2.2102   ...         47.94          47.94          10.84 
14997         3.0835        -0.884          1.4505        -2.2091   ...         47.94          47.94          10.857
14998         3.0833        -0.8844         1.4508        -2.208    ...         47.94          47.94          10.89 
14999         3.0831        -0.8847         1.4511        -2.2071   ...         47.94          47.94          11.29 
15000         3.0829        -0.885          1.4514        -2.2062   ...         47.94          47.955         11.69 

 name            f_y            f_z
 role   unused_float   unused_float
    0         6.9           -7.33  
    1         6.7218        -7.4427
    2         6.5436        -7.5555
    3         6.4533        -7.65  
    4         6.6267        -7.69  
             ...            ...    
14996        -1.41          16.14  
14997        -1.52          15.943 
14998        -1.74          15.55  
14999        -1.4601        15.743 
15000        -1.1801        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
name: data_all
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/robot/data_all/

### 1.2 Prepare data

The force vector consists of three component (*f_x*, *f_y* and *f_z*), meaning that we have three targets. For this comparison, we only predict the first component (*f_x*). 

Also, we want to speed things up a little, so we only use 10 columns. A previous analysis has revealed that the predictive power is mainly extracted from these 10 columns:

In [5]:
only_use = ["30", "34", "37", "38", "4", "59", "61", "7", "77", "78"]

data_all.set_role(["f_x"], getml.data.roles.target)
data_all.set_role(only_use, getml.data.roles.numerical)

This is what the data set looks like:

In [6]:
data_all

name       f_x          30          34          37   ...            104            105            106            f_y
 role    target   numerical   numerical   numerical   ...   unused_float   unused_float   unused_float   unused_float
    0   -11.03      -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.9   
    1   -10.848     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.7218
    2   -10.666     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.5436
    3   -10.507     -1.2042     -0.3273     -1.0191   ...         47.834         47.955         47.971         6.4533
    4   -10.413     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.6267
         ...        ...         ...         ...                   ...            ...            ...           ...    
14996    10.84      -2.6343     -0.8836     -1.5591   ...         47.803         47.94          47.94         -1.41  
14997    10.857     -2.6331     -0.8841     -1.5596   ...         47.803         47.94          47.94         -1.52  
14998    10.89      -2.6321     -0.8844     -1.56     ...         47.803         47.94          47.94         -1.74  
14999    11.29      -2.6311     -0.8848     -1.5604   ...         47.803         47.94          47.94         -1.4601
15000    11.69      -2.6302     -0.8851     -1.5607   ...         47.803         47.94          47.955        -1.1801

 name            f_z
 role   unused_float
    0        -7.33  
    1        -7.4427
    2        -7.5555
    3        -7.65  
    4        -7.69  
             ...    
14996        16.14  
14997        15.943 
14998        15.55  
14999        15.743 
15000        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
name: data_all
type: getml.DataFrame
url: http://localhost:1709/#/getdataframe/robot/data_all/

### 1.3 Separate data into a training and testing set

We also want to separate the data set into a training and testing set. We do so by using the first 10,500 measurements for training and then using the remainder for testing.

In [7]:
split = getml.data.split.time(data_all, "rowid", test=10500)
split

,
0,train
1,train
2,train
3,train
4,train
,...


In [8]:
time_series = getml.data.TimeSeries(
    population=data_all,
    split=split,
    time_stamps="rowid",
    lagged_targets=False,
    memory=30,
)

time_series

data model

  population:
    columns:
    - 30: numerical
    - 34: numerical
    - 37: numerical
    - 38: numerical
    - 4: numerical
    - ...

    joins:
    - right: 'data_all'
      time_stamps: (population.rowid, data_all.rowid)
      relationship: 'many-to-many'
      memory: 30
      lagged_targets: False

  data_all:
    columns:
    - 30: numerical
    - 34: numerical
    - 37: numerical
    - 38: numerical
    - 4: numerical
    - ...


container

  population
      subset   name        rows   type
  0   test     data_all    4501   View
  1   train    data_all   10500   View

  peripheral
      name        rows   type
  0   data_all   15001   View

## 2. Predictive modeling

### 2.1 Propositionalization with getML's FastProp

In [9]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
)

In [10]:
pipe_fp_fl = getml.pipeline.Pipeline(
    data_model=time_series.data_model,
    feature_learners=[fast_prop],
    tags=["feature learning", "fastprop"],
)

In [11]:
pipe_fp_fl.check(time_series.train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


OK.


In [12]:
benchmark = Benchmark()

In [13]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)

fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...


Staging...
[========================================] 100%


OK.


Staging...
[========================================] 100%

FastProp: Trying 134 features...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:0.020292



Staging...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




In [14]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")



Staging...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




In [15]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [16]:
pipe_fp_pr.check(fastprop_train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_123' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_125' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_128' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting 

In [17]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...


Staging...
[========================================] 100%


WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_123' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_125' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_128' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOU

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

url: http://localhost:1709/#/getpipeline/robot/5z1iaI/0/

In [18]:
pipe_fp_pr.score(fastprop_test)



Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-11-03 19:52:59,fastprop_train,f_x,0.4383,0.5764,0.9963
1,2021-11-03 19:52:59,fastprop_test,f_x,0.5515,0.7236,0.9951


### 2.2 Propositionalization with featuretools

In [19]:
data_train = time_series.train.population.to_df("data_train")
data_test = time_series.test.population.to_df("data_test")

In [20]:
dfs_pandas = {}

for df in [data_train, data_test, data_all]:
    dfs_pandas[df.name] = df.to_pandas()
    delete_columns = [
        col for col in dfs_pandas[df.name].columns if col not in only_use + ["f_x"]
    ]
    for col in delete_columns:
        del dfs_pandas[df.name][col]
    dfs_pandas[df.name]["id"] = 1
    dfs_pandas[df.name]["ds"] = pd.to_datetime(
        np.arange(0, dfs_pandas[df.name].shape[0]), unit="s"
    )

In [21]:
dfs_pandas["data_train"]

,30,34,37,38,4,59,61,7,77,78,f_x,id,ds
0,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082791,0.78597,-1.0191,0.082782,-1.4094,-11.0300,1,1970-01-01 00:00:00
1,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082800,0.78592,-1.0191,0.082782,-1.4094,-10.8480,1,1970-01-01 00:00:01
2,-1.2042,-0.32737,-1.0191,-6.0205,-0.32737,0.082786,0.78594,-1.0191,0.082782,-1.4094,-10.6660,1,1970-01-01 00:00:02
3,-1.2042,-0.32734,-1.0191,-6.0205,-0.32737,0.082755,0.78599,-1.0191,0.082782,-1.4094,-10.5070,1,1970-01-01 00:00:03
4,-1.2042,-0.32736,-1.0191,-6.0205,-0.32737,0.082782,0.78597,-1.0191,0.082782,-1.4094,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.1446,-0.37311,-1.0486,-5.9532,-0.37326,0.087343,0.90793,-1.0488,0.087468,-1.4162,-9.7673,1,1970-01-01 02:54:55
10496,-1.1349,-0.37103,-1.0472,-5.9564,-0.37108,0.087241,0.90199,-1.0474,0.087274,-1.4160,-9.9200,1,1970-01-01 02:54:56
10497,-1.1255,-0.36889,-1.0458,-5.9596,-0.36896,0.087055,0.89618,-1.0460,0.087082,-1.4158,-9.7743,1,1970-01-01 02:54:57
10498,-1.1163,-0.36680,-1.0444,-5.9627,-0.36689,0.086907,0.89034,-1.0447,0.086893,-1.4155,-8.6109,1,1970-01-01 02:54:58


In [22]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(seconds=0),
    memory=pd.Timedelta(seconds=15),
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [23]:
with benchmark("featuretools"):
    featuretools_train = ft_builder.fit(dfs_pandas["data_train"])

featuretools_test = ft_builder.transform(dfs_pandas["data_test"])

featuretools: Trying features...


/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:309: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 181 features...
Time taken: 0h:5m:54.939827



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:309: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [24]:
featuretools_train

,MIN(peripheral.37),MIN(peripheral.7),FIRST(peripheral.37),FIRST(peripheral.7),MEDIAN(peripheral.37),MEDIAN(peripheral.7),MEAN(peripheral.37),MEAN(peripheral.7),SUM(peripheral.37),SUM(peripheral.7),...,MEAN(peripheral.index),MIN(peripheral.index),FIRST(peripheral._featuretools_index),FIRST(peripheral.index),TIME_SINCE_FIRST(peripheral.ds),COUNT(peripheral),SUM(peripheral.id),f_x,id,ds
_featuretools_index,,,,,,,,,,,,,,,,,,,,,
0,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-1.0191,-1.0191,...,0.0,0.0,0,0,1.635969e+09,1,1.0,-11.0300,1,1970-01-01 00:00:00
1,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-2.0382,-2.0382,...,1.5,1.0,1,1,1.635969e+09,2,2.0,-10.8480,1,1970-01-01 00:00:01
2,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-3.0573,-3.0573,...,4.0,3.0,3,3,1.635969e+09,3,3.0,-10.6660,1,1970-01-01 00:00:02
3,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-4.0764,-4.0764,...,7.5,6.0,6,6,1.635969e+09,4,4.0,-10.5070,1,1970-01-01 00:00:03
4,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-5.0955,-5.0955,...,12.0,10.0,10,10,1.635969e+09,5,5.0,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.0716,-1.0721,-1.0716,-1.0721,-1.0596,-1.0596,-1.059793,-1.059927,-15.8969,-15.8989,...,157327.0,157320.0,157320,157320,1.635959e+09,15,15.0,-9.7673,1,1970-01-01 02:54:55
10496,-1.0698,-1.0702,-1.0698,-1.0702,-1.0580,-1.0580,-1.058167,-1.058280,-15.8725,-15.8742,...,157342.0,157335.0,157335,157335,1.635959e+09,15,15.0,-9.9200,1,1970-01-01 02:54:56
10497,-1.0681,-1.0684,-1.0681,-1.0684,-1.0565,-1.0563,-1.056567,-1.056667,-15.8485,-15.8500,...,157357.0,157350.0,157350,157350,1.635959e+09,15,15.0,-9.7743,1,1970-01-01 02:54:57


In [25]:
roles = {
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=roles
)

In [26]:
df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

In [27]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [28]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:5.797169



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

url: http://localhost:1709/#/getpipeline/robot/jAXgnf/0/

In [29]:
pipe_ft_pr.score(df_featuretools_test)



Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-11-03 20:01:35,featuretools_train,f_x,0.436,0.582,0.9962
1,2021-11-03 20:01:35,featuretools_test,f_x,0.5713,0.7528,0.9946


### 2.3 Propositionalization with tsfresh

In [30]:
tsfresh_builder = TSFreshBuilder(
    num_features=200,
    memory=15,
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [31]:
with benchmark("tsfresh"):
    tsfresh_train = tsfresh_builder.fit(dfs_pandas["data_train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["data_test"])

Feature Extraction: 100%|██████████| 20/20 [00:36<00:00,  1.83s/it]


Selecting the best out of 120 features...
Time taken: 0h:1m:21.462958



Feature Extraction: 100%|██████████| 20/20 [00:17<00:00,  1.15it/s]


In [32]:
roles = {
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=roles
)

df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=roles
)

In [33]:
df_tsfresh_train.set_role(df_tsfresh_train.roles.unused, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.roles.unused, getml.data.roles.numerical)

In [34]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [35]:
pipe_tsf_pr.check(df_tsfresh_train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


OK.


In [36]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...


Staging...
[========================================] 100%


OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:7.218033



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

url: http://localhost:1709/#/getpipeline/robot/2bNjLX/0/

In [37]:
pipe_tsf_pr.score(df_tsfresh_test)



Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-11-03 20:03:48,tsfresh_train,f_x,0.4873,0.6564,0.9952
1,2021-11-03 20:03:48,tsfresh_test,f_x,0.5939,0.7857,0.9939


## 3. Comparison

In [39]:
num_features = dict(
    fastprop=123,
    featuretools=177,
    tsfresh=110,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
    benchmark.runtimes["tsfresh"] / num_features["tsfresh"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[
            benchmark.runtimes["fastprop"],
            benchmark.runtimes["featuretools"],
            benchmark.runtimes["tsfresh"],
        ],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
            benchmark.runtimes["tsfresh"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [40]:
comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:00:00.114531,123,1074.113856,1.00000,1.000000,0.995059,0.723603,0.551513
featuretools,0 days 00:05:54.940933,177,0.498675,3099.08176,2153.937701,0.994614,0.752845,0.571329
tsfresh,0 days 00:01:21.463210,110,1.350302,711.27651,795.461869,0.993929,0.785746,0.593878


In [41]:
comparison.to_csv("comparisons/robot.csv")

## Why is FastProp so fast?

First, FastProp hugely benefits from getML's custom-built C++-native in-memory database engine. The engine is highly optimized for working with relational data structures and makes use of information about the relational structure of the data to efficiently store and carry out computations on such data. This matters in particular for time series where we [relate the current observation to a certain number of observations from the past](https://docs.getml.com/latest/user_guide/data_model/data_model.html#time-series): Other libraries have to deal explicitly with this inherent structure of (multivariate) time series; and such explicit transformations are costly, in terms of consumption of both, memory and computational resources. All operations on data stored in getML's engine benefit from implementations in modern C++. Further, we are taking advantage of functional design patterns where all column-based operations are evaluated lazily. So, for example, aggregations are carried out only on rows that matter (taking into account even complex conditions that might span multiple tables in the relational model). Duplicate operations are reduced to a bare minimum by keeping track of the relational data model. In addition to the mere advantage in performance, FastProp, by building on an abstract data model, also has an edge in memory consumption based on the abstract database design, the reliance on efficient storage patterns (utilizing pointers and indices) for concrete data, and by taking advantage of functional design patterns and lazy computations. This allows working with data sets of substantial size even without falling back to distributed computing models.

# Next Steps

If you are interested in further real-world applications of getML, visit the [notebook section on getml.com](https://notebooks.getml.com/). If you want to gain a deeper understanding about our notebooks' contents or download the code behind the notebooks, have a look at the [getml-demo repository](https://github.com/getml/getml-demo/). Here, you can also find [futher benchmarks of getML](https://github.com/getml/getml-demo/#benchmarks).

Want to try out without much hassle? Just head to [try.getml.com](https://try.getml.com) to launch an instance of getML directly in your browser.

Further, here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Annotating data within getML's data frames](https://docs.getml.com/latest/user_guide/annotating_data/annotating_data.html),
* [Defining your relational structure through getML's abstract data model](https://docs.getml.com/latest/user_guide/data_model/data_model.html), or
* [An introduction to feature learning](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html).

# Get in contact

If you have any questions, just write us an [email](https://getml.com/contact/lets-talk/). Prefer a private demo of getML for your team? Just [contact us](https://getml.com/contact/lets-talk/) to arrange an introduction to getML.